In [4]:
import json
from datasets import load_dataset
from pyensign.events import Event
from pyensign.ensign import Ensign

def load_events(split):
    for record in load_dataset("imdb", split=split, streaming=True):
        data = {
            "text": record["text"],
            "label": record["label"],
            "split": split,
        }
        yield Event(json.dumps(data).encode("utf-8"), mimetype="application/json", schema_name="reviews-raw", schema_version="0.1.0")

ensign = Ensign(client_id="gxfoGlYfcyZZxrCwLSnyEWLZDekmhRQF", client_secret="UaFmMblmfgmUacTFONVm3NDL6JNH25bJSi6rQEd1eXnh0cwnp1hMr44NWqpjeCZc")
TEXT_TOPIC = "movie-reviews-text"
for event in load_events("train"):
    await ensign.publish(TEXT_TOPIC, event)

for event in load_events("test"):
    await ensign.publish(TEXT_TOPIC, event)

for event in load_events("unsupervised"):
    await ensign.publish(TEXT_TOPIC, event)

In [7]:
from pyensign.ml.dataframe import DataFrame

cursor = await ensign.query("SELECT * FROM movie-reviews-text")
df = await DataFrame.from_events(cursor)
df.head()

,label,split,text
066eztdvp8004ryw,0,train,I rented I AM CURIOUS-YELLOW from my video sto...
066eztdvq0004ryx,0,train,"""I Am Curious: Yellow"" is a risible and preten..."
066eztdvq8004ryy,0,train,If only to avoid making this type of film in t...
066eztdvqg004ryz,0,train,This film was probably inspired by Godard's Ma...
066eztdvqr004rz0,0,train,"Oh, brother...after hearing about this ridicul..."


In [9]:
print("train", len(df[df["split"] == "train"]))
print("test", len(df[df["split"] == "test"]))
print("unsupervised", len(df[df["split"] == "unsupervised"]))

25000
25000
50000
